In [ ]:

import openeo
import json
import pandas as pd
from openeo.rest.conversions import timeseries_json_to_pandas

In [7]:
def connect(url):
    """
    This function connects to the openEO backend and returns the connection object.
    Parameters: url --> String
    Returns: connection object
    """
    print(openeo.client_version())
    connection = openeo.connect(url = url).authenticate_oidc()
    return connection

In [4]:
connection = openeo.connect(url = "openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=TOPH-UPKD 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


In [5]:
connection.list_collection_ids()

['SENTINEL3_OLCI_L1B',
 'SENTINEL3_SLSTR',
 'SENTINEL_5P_L2',
 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON1',
 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON2',
 'COPERNICUS_PLANT_PHENOLOGY_INDEX',
 'ESA_WORLDCOVER_10M_2020_V1',
 'ESA_WORLDCOVER_10M_2021_V2',
 'COPERNICUS_VEGETATION_INDICES',
 'SENTINEL2_L1C',
 'SENTINEL2_L2A',
 'SENTINEL1_GRD',
 'COPERNICUS_30',
 'LANDSAT8_L2',
 'SENTINEL3_SYN_L2_SYN',
 'SENTINEL3_SLSTR_L2_LST',
 'SENTINEL1_GLOBAL_MOSAICS']

In [5]:
# connection.list_jobs()

In [ ]:
# To delete jobs
job = connection.job("j-241021aacc67492ab6a60522766f4a5b")
job.delete() 
#test

In [88]:
# To check status
job_id = "j-2410099384c34289a54bda4a953ab5fa"
job = connection.job(job_id)
job.status()
job.start_and_wait()

In [10]:
import os
file_path = "D:\\code\\yield_prediction\\data\\Vidisha_Bhaumrasa_Village_Boundary.geojson"
file_path = os.path.dirname(os.getcwd()) + "\\data\\vidisha_district_boundary.geojson"
with open(file_path, 'r') as file:
    features = json.load(file)

In [11]:
# Load raw collection data
sentinel2_cube = connection.load_collection(
    "COPERNICUS_VEGETATION_INDICES",
    spatial_extent=features,
    temporal_extent = ["2024-07-25", "2024-09-20"],
    bands=["B02", "B03", "B04", "B05", "B08", "B12", "SCL"],
)
# Use the scene classification layer to mask out non-vegetation pixels
scl = sentinel2_cube.band("SCL")
cloud_mask = (scl == 3) | (scl == 8) | (scl == 9)
cloud_mask = cloud_mask.resample_cube_spatial(sentinel2_cube)
cube_masked = sentinel2_cube.mask(cloud_mask)

# Extract spectral bands and calculate indices with the "band math" feature
blue = cube_masked.band("B02") * 0.0001
red = cube_masked.band("B04") * 0.0001
nir = cube_masked.band("B08") * 0.0001
re = cube_masked.band("B05") * 0.0001
green = cube_masked.band("B03") * 0.0001
swir = cube_masked.band("B12") * 0.0001

evi = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)
ndvi =  (nir - red) / (nir + red)
reci = (nir/red) - 1 
ndre = (nir - re)/(nir + re)
savi = ((nir - red)/(nir + red + 0.5)) * 1.5
gcvi = (nir / green) - 1
lswi = (nir - swir) / (nir + swir)
ndwi = (green - nir)/(green + nir)

evi_masked = evi.mask(scl != 4)
ndvi_masked = ndvi.mask(scl != 4)
reci_masked = reci.mask(scl != 4)
ndre_masked = ndre.mask(scl != 4)
savi_masked = savi.mask(scl != 4)
gcvi_masked = gcvi.mask(scl != 4)
lswi_masked = lswi.mask(scl != 4)
ndwi_masked = ndwi.mask(scl != 4)

blue_masked = blue.mask(scl != 4)
red_masked = red.mask(scl != 4)
nir_masked = nir.mask(scl != 4)
re_masked = re.mask(scl != 4)
green_masked = green.mask(scl != 4)
swir_masked = swir.mask(scl != 4)


ValueError: Invalid band name/index 'B02'. Valid names: ['NDVI', 'PPI', 'LAI', 'FAPAR', 'QFLAG2']

In [96]:
evi_composite = evi_masked.mean_time()
ndvi_composite = ndvi_masked.mean_time()
reci_composite = reci_masked.mean_time()
ndre_composite = ndre_masked.mean_time()
savi_composite = savi_masked.mean_time()
gcvi_composite = gcvi_masked.mean_time()
lswi_composite = lswi_masked.mean_time()
ndwi_composite = ndwi_masked.mean_time()

blue_composite = blue_masked.mean_time()
red_composite = red_masked.mean_time()
nir_composite = nir_masked.mean_time()
re_composite = re_masked.mean_time()
green_composite = green_masked.mean_time()
swir_composite = swir_masked.mean_time()

In [97]:
evi_cube = evi_composite.save_result(format="GTiff")
evi_job = evi_cube.create_job(title="EVI_Vidisha_Bhaumrasa")
ndvi_cube = ndvi_composite.save_result(format="GTiff")
ndvi_job = ndvi_cube.create_job(title = "NDVI_Vidisha_Bhaumrasa")
reci_cube = reci_composite.save_result(format="GTiff")
reci_job = reci_cube.create_job(title="RECI_Vidisha_Bhaumrasa")
ndre_cube = ndre_composite.save_result(format="GTiff")
ndre_job = ndre_cube.create_job(title = "NDRE_Vidisha_Bhaumrasa")
savi_cube = savi_composite.save_result(format="GTiff")
savi_job = savi_cube.create_job(title="SAVI_Vidisha_Bhaumrasa")
gcvi_cube = gcvi_composite.save_result(format="GTiff")
gcvi_job = gcvi_cube.create_job(title = "GCVI_Vidisha_Bhaumrasa")
lswi_cube = lswi_composite.save_result(format="GTiff")
lswi_job = lswi_cube.create_job(title="LSWI_Vidisha_Bhaumrasa")
ndwi_cube = ndwi_composite.save_result(format="GTiff")
ndwi_job = ndwi_cube.create_job(title = "NDWI_Vidisha_Bhaumrasa")

blue_cube = blue_composite.save_result(format="GTiff")
blue_job = blue_cube.create_job(title = "Blue_Vidisha_Bhaumrasa")
red_cube = red_composite.save_result(format="GTiff")
red_job = red_cube.create_job(title = "Red_Vidisha_Bhaumrasa")
nir_cube = nir_composite.save_result(format="GTiff")
nir_job = nir_cube.create_job(title = "NIR_Vidisha_Bhaumrasa")
re_cube = re_composite.save_result(format="GTiff")
re_job = re_cube.create_job(title = "RE_Vidisha_Bhaumrasa")
green_cube = green_composite.save_result(format="GTiff")
green_job = green_cube.create_job(title = "Green_Vidisha_Bhaumrasa")
swir_cube = swir_composite.save_result(format="GTiff")
swir_job = swir_cube.create_job(title = "SWIR_Vidisha_Bhaumrasa")

In [98]:
evi_job_id = evi_job.job_id
print(f"EVI Job ID is {evi_job_id}")
ndvi_job_id = ndvi_job.job_id
print(f"NDVI Job ID is {ndvi_job_id}")
reci_job_id = reci_job.job_id
print(f"RECI Job ID is {reci_job_id}")
ndre_job_id = ndre_job.job_id
print(f"NDRE Job ID is {ndre_job_id}")
savi_job_id = savi_job.job_id
print(f"SAVI Job ID is {savi_job_id}")
gcvi_job_id = gcvi_job.job_id
print(f"GCVI Job ID is {gcvi_job_id}")
lswi_job_id = lswi_job.job_id
print(f"LSWI Job ID is {lswi_job_id}")
ndwi_job_id = ndwi_job.job_id
print(f"NDWI Job ID is {ndwi_job_id}")

blue_job_id = blue_job.job_id
print(f"Blue Job ID is {blue_job_id}")
red_job_id = red_job.job_id
print(f"Red Job ID is {red_job_id}")
nir_job_id = nir_job.job_id
print(f"NIR Job ID is {nir_job_id}")
re_job_id = re_job.job_id
print(f"RE Job ID is {re_job_id}")
green_job_id = green_job.job_id
print(f"Green Job ID is {green_job_id}")
swir_job_id = swir_job.job_id
print(f"SWIR Job ID is {swir_job_id}")


EVI Job ID is j-241022adb7124af69c719012142ee0fa
NDVI Job ID is j-24102249403d468fb5d6fc49551657b1
RECI Job ID is j-24102270254b443285c0e538ddbe24b1
NDRE Job ID is j-24102218c19942cd9fab3a2e41085abe
SAVI Job ID is j-2410226430c04bb9807f2d17d5f1a9fa
GCVI Job ID is j-2410228b25ee409193f12863259e26ec
LSWI Job ID is j-241022461dda4aaa972e40def01302ee
NDWI Job ID is j-241022c70cf143759f1f6b965a339e01
Blue Job ID is j-241022baec7d451681fb6fa87385b77b
Red Job ID is j-2410229351ae41ef98bf4f85d1bb4640
NIR Job ID is j-24102251bea04f9f907ec3e10417fd8f
RE Job ID is j-24102243e1094672828df6ae1476dc1a
Green Job ID is j-241022d8b36f49da92b87d629f8aa689
SWIR Job ID is j-2410229810c849649bfc17f243ec15e2


In [18]:
job_id = 'j-2410229810c849649bfc17f243ec15e2'
job = connection.job(job_id)
job.status()
results =job.get_results()
results.download_file("data/out/Vidisha_Bhaumrasa/swir_Bhaumrasa.tiff")

WindowsPath('data/out/Vidisha_Bhaumrasa/swir_Bhaumrasa.tiff')

In [99]:
evi_job.start()
evi_job.status()
evi_job.start_and_wait()

ndvi_job.start()
ndvi_job.status()
ndvi_job.start_and_wait()

reci_job.start()
reci_job.status()
reci_job.start_and_wait()

ndre_job.start()
ndre_job.status()
ndre_job.start_and_wait()

savi_job.start()
savi_job.status()
savi_job.start_and_wait()

gcvi_job.start()
gcvi_job.status()
gcvi_job.start_and_wait()

lswi_job.start()
lswi_job.status()
lswi_job.start_and_wait()

ndwi_job.start()
ndwi_job.status()
ndwi_job.start_and_wait()

blue_job.start()
blue_job.status()
blue_job.start_and_wait()

red_job.start()
red_job.status()
red_job.start_and_wait()

nir_job.start()
nir_job.status()
nir_job.start_and_wait()

re_job.start()
re_job.status()
re_job.start_and_wait()

green_job.start()
green_job.status()
green_job.start_and_wait()

swir_job.start()
swir_job.status()
swir_job.start_and_wait()


0:00:00 Job 'j-241022adb7124af69c719012142ee0fa': send 'start'
0:00:13 Job 'j-241022adb7124af69c719012142ee0fa': created (progress 0%)
0:00:18 Job 'j-241022adb7124af69c719012142ee0fa': created (progress 0%)
0:00:24 Job 'j-241022adb7124af69c719012142ee0fa': created (progress 0%)
0:00:33 Job 'j-241022adb7124af69c719012142ee0fa': created (progress 0%)
0:00:43 Job 'j-241022adb7124af69c719012142ee0fa': created (progress 0%)
0:00:55 Job 'j-241022adb7124af69c719012142ee0fa': running (progress N/A)
0:01:11 Job 'j-241022adb7124af69c719012142ee0fa': running (progress N/A)
0:01:30 Job 'j-241022adb7124af69c719012142ee0fa': running (progress N/A)
0:01:54 Job 'j-241022adb7124af69c719012142ee0fa': running (progress N/A)
0:02:25 Job 'j-241022adb7124af69c719012142ee0fa': running (progress N/A)
0:03:02 Job 'j-241022adb7124af69c719012142ee0fa': finished (progress 100%)
0:00:00 Job 'j-24102249403d468fb5d6fc49551657b1': send 'start'
0:00:13 Job 'j-24102249403d468fb5d6fc49551657b1': created (progress 0%)
0:

<BatchJob job_id='j-2410229810c849649bfc17f243ec15e2'>

## Weather Data

In [3]:
# NCML
# Import Statements
import pandas as pd
import imdlib as imd

In [ ]:
# IMD Data - Lat, Lon for loc, Run if csv files are not downloaded already
Lat = 26.92 # Haseran Tehsil UP - 26.925409596889427, 79.57995647924646, Chibramau Tehsil - 27.15, 79.49 
Lon = 79.57 
start_date = "2022-06-01"
end_date = "2022-11-31"
var_type = 'tmax' # rain. tmin, tmax
file_dir='../data'
data = imd.get_real_data(var_type, start_date, end_date, file_dir)
data.to_csv(f'imd_Haseran_{var_type}_{Lat}_{Lon}.csv', Lat, Lon, file_dir)

In [6]:
connection.list_jobs()

[{'created': '2024-09-05T09:33:59Z',
  'id': 'j-24090574d0d74674a0a661f8996f2e7f',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-09-05T09:38:21Z'},
 {'created': '2024-09-27T09:03:59Z',
  'id': 'j-240927cfbf484375a7d3e6475bbc45b0',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-09-27T11:48:07Z'},
 {'created': '2024-09-27T10:12:06Z',
  'id': 'j-2409279331f34d85a90da06585faad4b',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-09-27T12:44:11Z'},
 {'created': '2024-09-28T11:09:41Z',
  'id': 'j-240928b108c9479799025a7c22f1a38e',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-09-28T13:48:13Z'},
 {'created': '2024-10-01T08:02:46Z',
  'id': 'j-241001f1ba444bf7a30e1fcb3f4e2873',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-01T10:34:40Z'},
 {'created': '2024-09-05T09:41:36Z',
  'id': 'j-240905f3e5f44239bd3a8461bc2ecdb2',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-09-05T09:49:27Z'},
 {'created': '2024-10-04T03:28:26Z',
  'id': 'j-241004df2a4e4df881eb7d37e87ca0d8',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-04T08:53:06Z'},
 {'created': '2024-10-07T14:04:29Z',
  'id': 'j-241007313ce74b779aedba81a034e104',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-07T16:11:09Z'},
 {'created': '2024-10-07T14:04:30Z',
  'id': 'j-24100742f4b846ac928ae7b323335646',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-07T18:02:07Z'},
 {'created': '2024-10-08T06:25:53Z',
  'id': 'j-241008b364a2416599756cfa2494dfa6',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T08:22:07Z'},
 {'created': '2024-10-08T06:25:54Z',
  'id': 'j-24100880730d425ab8258fdff08f6e05',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T10:16:08Z'},
 {'created': '2024-10-08T06:25:57Z',
  'id': 'j-2410083b695c4cab839bb3dedd2c9e60',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T15:54:16Z'},
 {'created': '2024-10-08T06:25:58Z',
  'id': 'j-2410083a9313465d9f67816b8aee5d4b',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T17:40:11Z'},
 {'created': '2024-10-08T06:25:55Z',
  'id': 'j-241008d7a91c49799a78c5af3cc226b3',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T12:08:07Z'},
 {'created': '2024-10-08T06:25:56Z',
  'id': 'j-241008ca4aea4d438e182b726a8dc0d0',
  'progress': 100,
  'status': 'finished',
  'updated': '2024-10-08T14:02:09Z'}]

In [31]:
job = connection.job('j-2410083a9313465d9f67816b8aee5d4b')

In [32]:
job.status()

'finished'

In [33]:
results = job.get_results()

In [34]:
results.download_file("data/out/gcvi_mean_vidisha.tiff")

WindowsPath('data/out/gcvi_mean_vidisha.tiff')

In [11]:
results.download_files("data/out")

[WindowsPath('data/out/openEO.tif'), WindowsPath('data/out/job-results.json')]

In [12]:
results.get_metadata()

{'assets': {'openEO.tif': {'href': 'https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2409279331f34d85a90da06585faad4b/results/assets/NTg2Nzc5NjUtNWQ5MC00YWQ5LTlkMmMtYTRkYjIyYmJkNzY5/6432d0be3c1610098544a156fbb02fc5/openEO.tif?expires=1728120216',
   'proj:bbox': [702220, 2447840, 829050, 2605460],
   'proj:epsg': 32642,
   'proj:shape': [15762, 12683],
   'raster:bands': [{'name': '1',
     'statistics': {'maximum': 1821586.5,
      'mean': 0.43853540012353,
      'minimum': -1322.8629150391,
      'stddev': 164.75032592348,
      'valid_percent': 61.16}}],
   'roles': ['data'],
   'title': 'openEO.tif',
   'type': 'image/tiff; application=geotiff'}},
 'description': 'Results for batch job j-2409279331f34d85a90da06585faad4b',
 'extent': {'spatial': {'bbox': [[70.98081432708541,
     22.12364957286676,
     72.18940809772997,
     23.52672101640311]]},
  'temporal': {'interval': [['2022-06-01T00:00:00Z',
     '2022-11-30T00:00:00Z']]}},
 'id': 'j-2409279331f34d85a90da06585faad4b

In [20]:
ndvi_job = connection.job('j-240928b108c9479799025a7c22f1a38e')

In [21]:
results = ndvi_job.get_results()

In [22]:
results.get_metadata()

{'assets': {'openEO.tif': {'href': 'https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-240928b108c9479799025a7c22f1a38e/results/assets/NTg2Nzc5NjUtNWQ5MC00YWQ5LTlkMmMtYTRkYjIyYmJkNzY5/1bc218afa49de9ce2002a8536b4bd8db/openEO.tif?expires=1728275268',
   'proj:bbox': [702220, 2447840, 829050, 2605460],
   'proj:epsg': 32642,
   'proj:shape': [15762, 12683],
   'raster:bands': [{'name': '1',
     'statistics': {'maximum': 667.00531005859,
      'mean': 0.58387855578753,
      'minimum': -665.32153320312,
      'stddev': 0.18702057771857,
      'valid_percent': 61.17}}],
   'roles': ['data'],
   'title': 'openEO.tif',
   'type': 'image/tiff; application=geotiff'}},
 'description': 'Results for batch job j-240928b108c9479799025a7c22f1a38e',
 'extent': {'spatial': {'bbox': [[70.98081432708541,
     22.12364957286676,
     72.18940809772997,
     23.52672101640311]]},
  'temporal': {'interval': [['2022-06-01T00:00:00Z',
     '2022-11-30T00:00:00Z']]}},
 'id': 'j-240928b108c9479799025a7c2

In [23]:
results.download_file("data/out/ndvi_mean.tiff")

WindowsPath('data/out/ndvi_mean.tiff')

In [ ]:
# To download Sentinel data as datacubes
url = "openeo.dataspace.copernicus.eu"
connection = connect(url)